In [1]:
import pandas as pd
import numpy as np
import urllib2
import os
import xlrd
from xml.etree import ElementTree
from lxml import objectify
import urllib
from pandas_datareader import data

# Quiz 1

### Question 1

The American Community Survey distributes downloadable data about United States communities. Download the 2006 microdata survey about housing for the state of Idaho using download.file() from here: 

https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2Fss06hid.csv 

and load the data into R. The code book, describing the variable names is here: 

https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2FPUMSDataDict06.pdf 

How many properties are worth $1,000,000 or more?

In [2]:
# read the url
url = "https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2Fss06hid.csv"
link = urllib2.urlopen(url)

# open the csv
data = pd.read_csv(link)

Property value attribute: VAL
Value worth more than $1000000 corresponds to code 24

In [3]:
data[data['VAL'] == 24].shape

(53, 188)

### Question 3

Download the Excel spreadsheet on Natural Gas Aquisition Program here: 

https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2FDATA.gov_NGAP.xlsx 

Read rows 18-23 and columns 7-15 into R and assign the result to a variable called:  
 dat  
What is the value of:

    sum(dat$Zip*dat$Ext,na.rm=T) 

In [4]:
%%bash
wget https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2FDATA.gov_NGAP.xlsx

--2015-11-11 18:46:46--  https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2FDATA.gov_NGAP.xlsx
Resolving d396qusza40orc.cloudfront.net (d396qusza40orc.cloudfront.net)... 54.230.4.126, 54.192.7.140, 54.230.4.254, ...
Connecting to d396qusza40orc.cloudfront.net (d396qusza40orc.cloudfront.net)|54.230.4.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16197 (16K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘getdata%2Fdata%2FDATA.gov_NGAP.xlsx’

     0K .......... .....                                      100%  148M=0s

2015-11-11 18:46:46 (148 MB/s) - ‘getdata%2Fdata%2FDATA.gov_NGAP.xlsx’ saved [16197/16197]



In [5]:
# To load the excel file, pandas requires the sheet name, xlrd serves the purpose
xls = xlrd.open_workbook(r'getdata%2Fdata%2FDATA.gov_NGAP.xlsx', on_demand=True)
sheet_name = xls.sheet_names()[0]

xls = pd.ExcelFile("getdata%2Fdata%2FDATA.gov_NGAP.xlsx")
df = xls.parse(sheet_name)

# xls = pd.read_excel("getdata%2Fdata%2FDATA.gov_NGAP.xlsx", 0) 
# 0 is default, read the 1st sheet, None, all sheets as a dictionary of DataFrames

In [6]:
dat = df.iloc[17:23, [6,11]]
dat = dat.dropna(axis=0)

In [7]:
print sum(dat.iloc[:, 0] * dat.iloc[:, 1])

36534720


In [8]:
os.remove("getdata%2Fdata%2FDATA.gov_NGAP.xlsx")

### Question 4

Read the XML data on Baltimore restaurants from here: 

https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2Frestaurants.xml 

How many restaurants have zipcode 21231?

In [9]:
%%bash
wget https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2Frestaurants.xml 

--2015-11-11 18:46:47--  https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2Frestaurants.xml
Resolving d396qusza40orc.cloudfront.net (d396qusza40orc.cloudfront.net)... 54.230.5.98, 54.192.7.146, 54.192.7.140, ...
Connecting to d396qusza40orc.cloudfront.net (d396qusza40orc.cloudfront.net)|54.230.5.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 695658 (679K) [text/xml]
Saving to: ‘getdata%2Fdata%2Frestaurants.xml’

     0K .......... .......... .......... .......... ..........  7%  658K 1s
    50K .......... .......... .......... .......... .......... 14% 1.44M 1s
   100K .......... .......... .......... .......... .......... 22% 1.13M 1s
   150K .......... .......... .......... .......... .......... 29% 1.43M 0s
   200K .......... .......... .......... .......... .......... 36% 1.42M 0s
   250K .......... .......... .......... .......... .......... 44% 1.50M 0s
   300K .......... .......... .......... .......... .......... 51% 1.47M 0s
   350K ..........

In [11]:
##### xmlparse = ElementTree.parse('getdata%2Fdata%2Frestaurants.xml')

xmlparse = objectify.parse('getdata%2Fdata%2Frestaurants.xml')

In [12]:
root = xmlparse.getroot()
root1 = root.getchildren()[0]
keys = ['name', 'zipcode', 'neighborhood', 'councildistrict', 'policedistrict']
df = pd.DataFrame(columns=keys)

In [13]:
index = 1
for element in root1.getchildren():
    dct = {}
    for sub_element in element.getiterator():
        dct[sub_element.tag] = sub_element.text
    row_s = pd.Series(dct)[keys]
    df.loc[index] = row_s
    index += 1

In [14]:
df = df.convert_objects(convert_numeric=True)

In [15]:
df[df.zipcode == 21231].shape[0]

127

In [16]:
os.remove("getdata%2Fdata%2Frestaurants.xml")

### Question 5

Read this data set into R and report the sum of the numbers in the fourth of the nine columns. 

https://d396qusza40orc.cloudfront.net/getdata%2Fwksst8110.for 

In [17]:
%%bash
wget https://d396qusza40orc.cloudfront.net/getdata%2Fwksst8110.for
head getdata%2Fwksst8110.for

 Weekly SST data starts week centered on 3Jan1990

                Nino1+2      Nino3        Nino34        Nino4
 Week          SST SSTA     SST SSTA     SST SSTA     SST SSTA
 03JAN1990     23.4-0.4     25.1-0.3     26.6 0.0     28.6 0.3
 10JAN1990     23.4-0.8     25.2-0.3     26.6 0.1     28.6 0.3
 17JAN1990     24.2-0.3     25.3-0.3     26.5-0.1     28.6 0.3
 24JAN1990     24.4-0.5     25.5-0.4     26.5-0.1     28.4 0.2
 31JAN1990     25.1-0.2     25.8-0.2     26.7 0.1     28.4 0.2
 07FEB1990     25.8 0.2     26.1-0.1     26.8 0.1     28.4 0.3


--2015-11-11 18:48:26--  https://d396qusza40orc.cloudfront.net/getdata%2Fwksst8110.for
Resolving d396qusza40orc.cloudfront.net (d396qusza40orc.cloudfront.net)... 54.192.7.140, 54.230.5.98, 54.192.4.210, ...
Connecting to d396qusza40orc.cloudfront.net (d396qusza40orc.cloudfront.net)|54.192.7.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79178 (77K) [application/octet-stream]
Saving to: ‘getdata%2Fwksst8110.for’

     0K .......... .......... .......... .......... .......... 64% 1.05M 0s
    50K .......... .......... .......                         100% 6.06M=0.05s

2015-11-11 18:48:27 (1.48 MB/s) - ‘getdata%2Fwksst8110.for’ saved [79178/79178]



In [18]:
names = ['Week', 'SST1+2', 'SSTA1+2', 'SST3', 'SSTA3', 'SST34', 'SSTA34', 'SST4', 'SSTA4']
colspecs = [(0,10),(15,19),(19,23),(28,32),(32,36),(41,45),(45,49),(54,58),(58,62)]

df = pd.read_fwf("getdata%2Fwksst8110.for", header=None, names=names, skiprows=4, colspecs=colspecs)

In [19]:
df.head()

,Week,SST1+2,SSTA1+2,SST3,SSTA3,SST34,SSTA34,SST4,SSTA4
0,03JAN1990,23.4,-0.4,25.1,-0.3,26.6,0.0,28.6,0.3
1,10JAN1990,23.4,-0.8,25.2,-0.3,26.6,0.1,28.6,0.3
2,17JAN1990,24.2,-0.3,25.3,-0.3,26.5,-0.1,28.6,0.3
3,24JAN1990,24.4,-0.5,25.5,-0.4,26.5,-0.1,28.4,0.2
4,31JAN1990,25.1,-0.2,25.8,-0.2,26.7,0.1,28.4,0.2


In [20]:
sum(df['SST3'])

32426.699999999986

In [21]:
os.remove("getdata%2Fwksst8110.for")

# Quiz 3

### Question 1

The American Community Survey distributes downloadable data about United States communities. Download the 2006 microdata survey about housing for the state of Idaho using download.file() from here: 

https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2Fss06hid.csv 

and load the data into R. The code book, describing the variable names is here: 

https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2FPUMSDataDict06.pdf 

Create a logical vector that identifies the households on greater than 10 acres who sold more than $10,000 worth of agriculture products. Assign that logical vector to the variable agricultureLogical. Apply the which() function like this to identify the rows of the data frame where the logical vector is TRUE. which(agricultureLogical) What are the first 3 values that result?

In [22]:
%%bash
wget https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2Fss06hid.csv

--2015-11-11 18:48:38--  https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2Fss06hid.csv
Resolving d396qusza40orc.cloudfront.net (d396qusza40orc.cloudfront.net)... 54.192.7.140, 54.192.4.210, 54.230.4.126, ...
Connecting to d396qusza40orc.cloudfront.net (d396qusza40orc.cloudfront.net)|54.192.7.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4246554 (4.0M) [text/csv]
Saving to: ‘getdata%2Fdata%2Fss06hid.csv’

     0K .......... .......... .......... .......... ..........  1%  777K 5s
    50K .......... .......... .......... .......... ..........  2% 1.10M 4s
   100K .......... .......... .......... .......... ..........  3% 1.49M 4s
   150K .......... .......... .......... .......... ..........  4% 5.19M 3s
   200K .......... .......... .......... .......... ..........  6% 1.69M 3s
   250K .......... .......... .......... .......... ..........  7% 2.35M 3s
   300K .......... .......... .......... .......... ..........  8% 5.11M 2s
   350K .......... ....

r:  

        data <- read.csv("getdata%2Fdata%2Fss06hid.csv")
        acre <- data['ACR'] == 3
        agri <- data['AGS'] == 6
        data['agricultureLOgical'] <- acre & agri
        which(data$agricultureLogical)

In [23]:
data = pd.read_csv("getdata%2Fdata%2Fss06hid.csv")
data.shape

(6496, 188)

In [24]:
acre = data['ACR'] == 3
agri = data.AGS == 6
data['agricultureLogical'] = acre & agri
data.shape

(6496, 189)

In [25]:
data[data['agricultureLogical']].index[:3] + 1

Int64Index([125, 238, 262], dtype='int64')

In [26]:
os.remove("getdata%2Fdata%2Fss06hid.csv")

### Question 2

Using the jpeg package read in the following picture of your instructor into R 

https://d396qusza40orc.cloudfront.net/getdata%2Fjeff.jpg 

Use the parameter native=TRUE. What are the 30th and 80th quantiles of the resulting data? (some Linux systems may produce an answer 638 different for the 30th quantile)

In [27]:
%%bash
wget https://d396qusza40orc.cloudfront.net/getdata%2Fjeff.jpg

--2015-11-11 18:48:50--  https://d396qusza40orc.cloudfront.net/getdata%2Fjeff.jpg
Resolving d396qusza40orc.cloudfront.net (d396qusza40orc.cloudfront.net)... 54.192.4.210, 54.230.5.98, 54.230.4.126, ...
Connecting to d396qusza40orc.cloudfront.net (d396qusza40orc.cloudfront.net)|54.192.4.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23849 (23K) [image/jpeg]
Saving to: ‘getdata%2Fjeff.jpg’

     0K .......... .......... ...                             100%  558K=0.04s

2015-11-11 18:48:51 (558 KB/s) - ‘getdata%2Fjeff.jpg’ saved [23849/23849]



r:  

    library(jpeg) # install jpeg if not exists
    data <- jpeg::readJPEG("get*.jpg", native = T)
    quantile(data, probs=c(0.3, 0.8), na.rm=T)

In [28]:
os.remove("getdata%2Fjeff.jpg")

### Question 3

Load the Gross Domestic Product data for the 190 ranked countries in this data set: 

https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2FGDP.csv 

Load the educational data from this data set: 

https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2FEDSTATS_Country.csv 

Match the data based on the country shortcode. How many of the IDs match? Sort the data frame in descending order by GDP rank (so United States is last). What is the 13th country in the resulting data frame? 

Original data sources: 
http://data.worldbank.org/data-catalog/GDP-ranking-table 
http://data.worldbank.org/data-catalog/ed-stats

** Convert the column to the proper before sorting the data **

In [29]:
%%bash
wget https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2FGDP.csv
wget https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2FEDSTATS_Country.csv

--2015-11-11 18:48:56--  https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2FGDP.csv
Resolving d396qusza40orc.cloudfront.net (d396qusza40orc.cloudfront.net)... 54.192.7.140, 54.230.4.254, 54.192.7.146, ...
Connecting to d396qusza40orc.cloudfront.net (d396qusza40orc.cloudfront.net)|54.192.7.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9630 (9.4K) [text/csv]
Saving to: ‘getdata%2Fdata%2FGDP.csv’

     0K .........                                             100%  130K=0.07s

2015-11-11 18:48:58 (130 KB/s) - ‘getdata%2Fdata%2FGDP.csv’ saved [9630/9630]

--2015-11-11 18:48:58--  https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2FEDSTATS_Country.csv
Resolving d396qusza40orc.cloudfront.net (d396qusza40orc.cloudfront.net)... 54.192.4.210, 54.230.5.98, 54.230.4.126, ...
Connecting to d396qusza40orc.cloudfront.net (d396qusza40orc.cloudfront.net)|54.192.4.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59792 (58K) [text/csv]
S

In [3]:
names = ['CountryCode', 'Rank', 'X', 'Economy', 'Size', 'X.1', 'X.2', 'X.3', 'X.4', 'X.5']
GDP = pd.read_csv("getdata%2Fdata%2FGDP.csv", sep=',', names=names, skiprows=5)

# clean the DataFrame
GDP.dropna(axis=1, how='all', inplace=True)
GDP.dropna(axis=0, subset=['CountryCode', 'Rank'], how='any', inplace=True)
GDP.head()

,CountryCode,Rank,Economy,Size,X.1
0,USA,1,United States,"16,244,600",NaN
1,CHN,2,China,"8,227,103",NaN
2,JPN,3,Japan,"5,959,718",NaN
3,DEU,4,Germany,"3,428,131",NaN
4,FRA,5,France,"2,612,878",NaN


In [4]:
FEDSTATS = pd.read_csv("getdata%2Fdata%2FEDSTATS_Country.csv")
FEDSTATS.dropna(axis=1, how='all', inplace=True)
FEDSTATS.head()

,CountryCode,Long Name,Income Group,Region,Lending category,Other groups,Currency Unit,Latest population census,Latest household survey,Special Notes,National accounts base year,National accounts reference year,System of National Accounts,SNA price valuation,Alternative conversion factor,PPP survey year,Balance of Payments Manual in use,External debt Reporting status,System of trade,Government Accounting concept,
0,ABW,Aruba,High income: nonOECD,Latin America & Caribbean,NaN,NaN,Aruban florin,2000,NaN,NaN,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Special,NaN,...
1,ADO,Principality of Andorra,High income: nonOECD,Europe & Central Asia,NaN,NaN,Euro,Register based,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,General,NaN,...
2,AFG,Islamic State of Afghanistan,Low income,South Asia,IDA,HIPC,Afghan afghani,1979,"MICS, 2003",Fiscal year end: March 20; reporting period fo...,2002/2003,NaN,NaN,VAB,NaN,NaN,NaN,Actual,General,Consolidated,...
3,AGO,People's Republic of Angola,Lower middle income,Sub-Saharan Africa,IDA,NaN,Angolan kwanza,1970,"MICS, 2001, MIS, 2006/07",NaN,1997,NaN,NaN,VAP,1991-96,2005,BPM5,Actual,Special,NaN,...
4,ALB,Republic of Albania,Upper middle income,Europe & Central Asia,IBRD,NaN,Albanian lek,2001,"MICS, 2005",NaN,NaN,1996,1993,VAB,NaN,2005,BPM5,Actual,General,Consolidated,...


In [32]:
GDP.shape

(190, 5)

In [11]:
Data_Merge = pd.merge(GDP, FEDSTATS, how='inner', on='CountryCode')
merge_key = ['CountryCode', 'Size', 'Short Name', 'Rank', 'Income Group']
Data_Merge = Data_Merge[merge_key]

# Reformat the Size column 123,456 -> 123456
Data_Merge.Size = Data_Merge.Size.str.replace(",", "")
Data_Merge.Size = Data_Merge.Size.convert_objects(convert_numeric=True)

# Resort the Data Frame by Size and CountryCode in an ascending order
Data_Merge.sort(['Size', 'CountryCode'], ascending=[True, True], inplace=True)

# Reset the index of the Data Frame
Data_Merge.set_index(np.arange(len(Data_Merge)) + 1, inplace=True)

In [12]:
Data_Merge.loc[13]

CountryCode                     KNA
Size                            767
Short Name      St. Kitts and Nevis
Rank                            178
Income Group    Upper middle income
Name: 13, dtype: object

In [8]:
os.remove("getdata%2Fdata%2FEDSTATS_Country.csv")
os.remove("getdata%2Fdata%2FGDP.csv")

R:  

    # read the data
    names <- c("CountryCode", "Ranking", "X", "Economy", "Size", "X.1", "X.2", "X.3", "X.4", "X.5")
    GDP <- read.csv("getdata%2Fdata%2FGDP.csv", skip = 4, col.names = names)
    FEDSTATS <- read.csv("getdata%2Fdata%2FEDSTATS_Country.csv")
    
    # merge the data
    Data_Merge <- merge(GDP, FEDSTATS, by="CountryCode")
    
    # convert the column $Size to numeric from factor
    Data_Merge$Size <- as.numeric(gsub(",", "", as.character(Data_Merge$Size)))
    
    merge_keys <- c("CountryCode","Size","Short.Name", "Ranking", "Income.Group")
    Data_Merge <- Data_Merge[merge_keys]
    
    Data_Merge[ order(Data_Merge$Size, Data_Merge$CountryCode), ]

### Question 4

What is the average GDP ranking for the "High income: OECD" and "High income: nonOECD" group?

In [13]:
# define a function that calculates the average ranking based on the "Income Group"
def ranker(df):
    df['avg'] = np.average(df.Rank)
    return df

Data_Merge.Rank = Data_Merge.Rank.convert_objects(convert_numeric=True)

# group the data by "Income Group" and calculate the average ranking
DF4 = Data_Merge.groupby("Income Group").apply(ranker)

# show the result
DF4 = DF4.loc[:, ["Income Group", "avg"]]
DF4.drop_duplicates()

,Income Group,avg
1,Lower middle income,107.703704
4,Upper middle income,92.133333
9,Low income,133.729730
30,High income: nonOECD,91.913043
68,High income: OECD,32.966667


R:

    # convert the $Ranking to numeric and remove the NA terms
    Data_Merge$Ranking <- as.numeric( levels(Data_Merge$Ranking) )[ Data_Merge$Ranking ]
    Data_Merge <- subset(Data_Merge, !is.na(Data_Merge$Ranking))
    
    ranker <- function(df) {
        df$avg <- ave(df$Ranking)
    }
    DF4 <- lapply( split(Data_Merge, Data_Merge$Income.Group), ranker )
    DF4["High income:OECD"]["avg"]
    DF4["High income:nonOECD"]["avg"]

### Question 5

Cut the GDP ranking into 5 separate quantile groups. Make a table versus Income.Group. How many countries are Lower middle income but among the 38 nations with highest GDP?

In [7]:
levels = 5

## Solution 1
Data_Merge['quant'] = pd.qcut(Data_Merge.Rank, levels)
Data_Merge['cat'] = pd.Categorical.from_array(Data_Merge.quant).labels
del Data_Merge['quant']

# the highest GDP has 'cat' equal to levels - 1
len(Data_Merge[ (Data_Merge.cat == (levels - 1)) & (Data_Merge['Income Group'] == 'Lower middle income') ])
######

## Solution 2
First38 = Data_Merge[ Data_Merge.Rank < Data_Merge.Rank.quantile(38.0/len(Data_Merge)) ]
len(First38[ First38['Income Group'] == "Lower middle income" ])
######

# Solution 3
# Bin the Data_Merge, this is BETTER than Solution 1
bins_35 = pd.qcut(Data_Merge.Rank, levels, retbins=True)[1]
Labels = range(levels)
Data_Merge['cat'] = pd.cut(Data_Merge.Rank, bins_35, labels=Labels)
pd.value_counts( (Data_Merge.cat == 0) & (Data_Merge["Income Group"] == "Lower middle income") )

False    185
True       5
dtype: int64

R:  

    num_levels <- 5
    quantile_rank <- quantile( Data_Merge$Ranking, probs=seq(0, 1, 1/num_levels) )
    cat <- cut( Data_Merge$Ranking, quantile_rank, include.lowest = TRUE )
    table <- split( Data_Merge$Income.Group, cat )[ [ levels(cat)[1] ] ]
    length( table[ table == "Lower middle income" ] )

# Quiz 4

### Question 1
The American Community Survey distributes downloadable data about United States communities. Download the 2006 microdata survey about housing for the state of Idaho using download.file() from here: 

https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2Fss06hid.csv 

and load the data into R. The code book, describing the variable names is here: 

https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2FPUMSDataDict06.pdf 

Apply strsplit() to split all the names of the data frame on the characters "wgtp". What is the value of the 123 element of the resulting list?

In [21]:
# Download the files:
url1 = "https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2Fss06hid.csv"

csv_file = url1.split('/')[-1]
if not os.path.exists(csv_file):
    urllib.urlretrieve(url1, csv_file)

In [22]:
data_41 = pd.read_csv(csv_file)
names_41 = data_41.columns.values.astype(str)

lst = []
for element in names_41:
    lst.append(element.split("wgtp")[-1])
    
lst[122]

'15'

In [23]:
os.remove(csv_file)

### Question 2  
Load the Gross Domestic Product data for the 190 ranked countries in this data set: 

https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2FGDP.csv 

Remove the commas from the GDP numbers in millions of dollars and average them. What is the average? 

Original data sources: http://data.worldbank.org/data-catalog/GDP-ranking-table

In [9]:
url1 = "https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2FGDP.csv"

response = urllib2.urlopen(url1)
df = pd.read_csv(response, names=names, skiprows=5)

df.Size = df.Size.str.replace(",", "")
df.Size = df.Size.convert_objects(convert_numeric=True)

np.average(df.Size[~df.Size.isnull()])

1453710.2205882352

R:  

    tmp <- strsplit(url1, "/")[[1]]
    dest_file <- tmp[length(tmp)]
    download.file(url1, tmp)
    
    names <- c("CountryCode", "Ranking", "X", "Economy", "Size", "X.1", "X.2", "X.3", "X.4", "X.5")
    df <- read.csv(tmp, skip=4, col.names=names)
    df$Size <- as.numeric(gsub(",", "", as.character(df$Size)))
    
    mean(df$Size, na.rm=T)

### Question 3
In the data set from Question 2 what is a regular expression that would allow you to count the number of countries whose name begins with "United"? Assume that the variable with the country names in it is named countryNames. How many countries begin with United?

In [10]:
target_str = "^United"
pd.value_counts(df.Economy.str.match(target_str))

False    225
True       3
dtype: int64

R:

    grep("^United", countryNames)

### Question 4
Load the Gross Domestic Product data for the 190 ranked countries in this data set: 

https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2FGDP.csv 

Load the educational data from this data set: 

https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2FEDSTATS_Country.csv 

Match the data based on the country shortcode. Of the countries for which the end of the fiscal year is available, how many end in June? 

In [14]:
url1 = "https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2FGDP.csv"
url2 = "https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2FEDSTATS_Country.csv"

response1 = urllib2.urlopen(url1)
response2 = urllib2.urlopen(url2)

GDP44 = pd.read_csv(response1, skiprows=5, names=names)
FEDSTATS44 = pd.read_csv(response2)

Data_Merge_44 = pd.merge(GDP44, FEDSTATS44, how='inner', on="CountryCode")
Data_Merge_44.shape

(224, 40)

In [17]:
crit = ( ~Data_Merge_44['Special Notes'].isnull() ) & ( Data_Merge_44['Special Notes'].str.match("(.*) end(.*)June (.*)") )
pd.value_counts(crit)

False    211
True      13
dtype: int64

In [18]:
Data_Merge_44.loc[crit, ["CountryCode", "Special Notes"]]

,CountryCode,Special Notes
11,AUS,Fiscal year end: June 30; reporting period for...
20,SWE,Fiscal year end: June 30; reporting period for...
37,EGY,Fiscal year end: June 30; reporting period for...
43,PAK,Fiscal year end: June 30; reporting period for...
55,KWT,Fiscal year end: June 30; reporting period for...
58,BGD,Fiscal year end: June 30; reporting period for...
60,PRI,Fiscal year end: June 30; reporting period for...
86,KEN,Fiscal year end: June 30; reporting period for...
105,UGA,Fiscal year end: June 30; reporting period for...
116,BWA,Fiscal year end: June 30; reporting period for...


R:

    

### Question 5
You can use the quantmod (http://www.quantmod.com/) package to get historical stock prices for publicly traded companies on the NASDAQ and NYSE. Use the following code to download data on Amazon's stock price and get the times the data was sampled.

    library(quantmod)
    amzn = getSymbols("AMZN",auto.assign=FALSE)
    sampleTimes = index(amzn) 

In [2]:
amzn = data.DataReader("AMZN", "yahoo")

R:

    length(grep("^2012", sampleTimes))